In [ ]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_predict
import numpy as np
from scikeras.wrappers import KerasClassifier
import os
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
# Organizing the data into tumor/nontumor and splitting it into train, val, and test sets

import os
import shutil
from sklearn.model_selection import train_test_split

os.makedirs('tumor', exist_ok=True)
os.makedirs('nontumor', exist_ok=True)

for tumor_dir in ['glioma_tumor', 'meningioma_tumor', 'pituitary_tumor']:
    for img in os.listdir(tumor_dir):
        shutil.move(os.path.join(tumor_dir, img), 'tumor')

for img in os.listdir('normal'):
    shutil.move(os.path.join('normal', img), 'nontumor')

shutil.rmtree('glioma_tumor')
shutil.rmtree('meningioma_tumor')
shutil.rmtree('pituitary_tumor')
shutil.rmtree('normal')

def split_data(source_dir, dest_dir, split_ratios=(0.8, 0.1, 0.1)):
    files = os.listdir(source_dir)
    train_files, temp_files = train_test_split(files, train_size=split_ratios[0], random_state=42)
    val_files, test_files = train_test_split(temp_files, train_size=split_ratios[1]/(split_ratios[1] + split_ratios[2]), random_state=42)
    
    for file in train_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(dest_dir, 'train', os.path.basename(source_dir), file))
    for file in val_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(dest_dir, 'val', os.path.basename(source_dir), file))
    for file in test_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(dest_dir, 'test', os.path.basename(source_dir), file))

for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(split, 'tumor'), exist_ok=True)
    os.makedirs(os.path.join(split, 'nontumor'), exist_ok=True)

split_data('tumor', '.')
split_data('nontumor', '.')

shutil.rmtree('tumor')
shutil.rmtree('nontumor')

In [ ]:
# Counting the number of files in each directory and printing the number of images per category in each dataset
import os

def count_files(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

for split in ['train', 'val', 'test']:
    tumor_count = count_files(os.path.join(split, 'tumor'))
    nontumor_count = count_files(os.path.join(split, 'nontumor'))
    print(f"{split.capitalize()} set - Tumor images: {tumor_count}, Nontumor images: {nontumor_count}")


In [ ]:
# Preprocessing the data using ImageDataGenerator with augmentation for training and normalization for validation/test

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    'val',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    'test',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)
